In [ ]:
import pandas as pd

In [ ]:
data_dir = 'data/'
operating_points_df = pd.read_csv(data_dir + 'linie-mit-betriebspunkten.csv',
                              delimiter=';')
train_tracks_df = pd.read_csv(data_dir + 'zugzahlen.csv',
                              delimiter=';')

In [ ]:
class Operating_point:
    def __init__(self, id_index, id_word, gps):
        self.id_index = id_index
        self.id_word = id_word
        self.gps = gps
        self.connections = []
        self.congestion = 0
        self.lines = []
        
    def add_connection(self, connection):
        self.connections.append(connection)
        
    def add_line(self, line_id):
        self.lines.append(line_id)
        
    def __str__(self):
        return f"{self.id_index:d} {self.id_word} {self.gps} " + \
               f"num_connections: {len(self.connections)} congestion: {self.congestion}"
    
class Connection:
    def __init__(self, from_op, to_op, trains, load):
        self.from_op = from_op
        self.to_op = to_op
        self.trains = trains
        self.load = load
        
    def __str__(self):
        return f"{self.from_op} {self.to_op} {self.trains} {self.load}"

In [ ]:
points = dict()

for index, row in operating_points_df.iterrows():
    id_word = row['Abbreviation of the operating point']
    
    points.setdefault(id_word,
                      Operating_point(index,
                                      id_word,
                                      (row['E'], row['N'])))
    points[id_word].add_line(row['LINIE'])

outside_ind = 9000

def add_outside(op):
    global outside_ind
    if op.id_word not in points:
        points.setdefault(op.id_word, op)
        outside_ind += 1
    
for index, row in train_tracks_df.iterrows():
    from_op = row['BP_Von_Abschnitt']
    to_op = row['BP_Bis_Abschnitt']
    if (from_op not in points.keys()): # connection enters Switzerland
        add_outside(Operating_point(outside_ind, from_op, (0,0)))
    if (to_op not in points.keys()): # connection leaves Switzerland
        add_outside(Operating_point(outside_ind, to_op, (0,0)))
    

    new_connection = Connection(from_op, row['BP_Bis_Abschnitt'],
                                row['Anzahl_Zuege'], row['Gesamtbelastung_Bruttotonnen'])
    points[from_op].add_connection(new_connection)
    
outside_ind

In [ ]:
x_coords = []
y_coords = []
ids = []
incidence_list = {}
for point in points.values():
    ids.append(point.id_index)
    x_coords.append(point.gps[0])
    y_coords.append(point.gps[1])
    incidence_list[point.id_index] = []
    
    for connection in point.connections:
        incidence_list[point.id_index].append(points[connection.to_op].id_index)
    
vis_data = {'ID': ids,
        'x': x_coords,
        'y': y_coords,
        'IL': incidence_list}  

vis_data

In [ ]:
from bokeh.plotting import figure, save

class Map:
    def __init__(self, ops, title):
        self.p = figure(title=title)
        self.ops = ops

    def plotnodes(self,):

        self.p.circle(x = self.ops['x'], y = self.ops['y'], size=10, color="red")

    def plotedges(self,):
        #loop ove rall points and plot incident lists
        for startingpoint in ops['ID']:
            #plot all edges in incidence list to points not covered yet
            #plot line to point if target> self index
            IL = self.ops['IL'][startingpoint]
            for target in IL:
                #if(target >startingpoint):
                #plot line
                xline = [ self.ops['x'][startingpoint], self.ops['x'][target] ]
                yline = [ self.ops['y'][startingpoint], self.ops['y'][target] ]
                self.p.line(xline, yline)
            #print("starting point idx", startingpoint, IL)

    def saveplot(self, name):
        save(obj=self.p, filename=name)

# Initialize the plot (p) and give it a title
p = figure(title="Operation Points")

# Create a list of x-coordinates
x_coords = [0,1,2,3,4]
y_coords = [5,4,1,2,0]
ID = [0,1,2,3,4]
IL = {0: [2,3], 1: [3,4,2], 2:[0,1], 3: [0,1] , 4:[1]}
ops = { 'ID': ID,
        'x': x_coords,
        'y': y_coords,
        'IL': IL }

ops = vis_data

map = Map(ops, "test")
map.plotnodes()
map.plotedges()
map.saveplot("test.html")